In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import requests
import re
import csv

In [2]:
# читаем файл в датафрейм
# escapechar: One-character string used to escape other characters.

draft = pd.read_csv('2020_for_publication/incomes_2020.csv', sep=',', escapechar='\\', 
                    dtype={'person_id': 'object', 'region_id': 'object'})
draft

/home/ksenia/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name
0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой
1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой
2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой
3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой
4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры"
131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры"
131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры"
131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры"


In [3]:
draft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131876 entries, 0 to 131875
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   size          131876 non-null  float64
 1   comment       411 non-null     object 
 2   section_id    131876 non-null  int64  
 3   income_year   131876 non-null  int64  
 4   original_fio  130182 non-null  object 
 5   full_name     16437 non-null   object 
 6   person_id     16437 non-null   object 
 7   position      123901 non-null  object 
 8   department    28247 non-null   object 
 9   office_name   131876 non-null  object 
 10  region_id     107504 non-null  object 
 11  region_name   107504 non-null  object 
 12  name          131876 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 13.1+ MB


In [4]:
# проверка не "съехали" ли ряды, 
# например в столбце год нет ничего кроме лет
draft.groupby('income_year').count()

,size,comment,section_id,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name
income_year,,,,,,,,,,,,
2020,131876,411,131876,130182,16437,16437,123901,28247,131876,107504,107504,131876


In [5]:
# случайно заметила, что в столбце position есть супруга (не должно быть)
# смотрим, сколько всего супруг и детей есть в этом стобце
draft[((draft['position']=='супруг') | (draft['position']=='супруга') | (draft['position']=='дочь') | (draft['position']=='сын'))]

,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name
43732,623243.06,NaN,3881581,2020,Никитин Алексей Владимирович,NaN,NaN,супруг,NaN,Территориальный орган Федеральной службы госуд...,79,Саратовская область,"Региональный, без структуры"
43734,294367.72,NaN,3881585,2020,Боим Мария Владимировна,NaN,NaN,супруга,NaN,Территориальный орган Федеральной службы госуд...,79,Саратовская область,"Региональный, без структуры"
43736,1188721.72,NaN,3881589,2020,Мальцев Антон Владимирович,NaN,NaN,супруг,NaN,Территориальный орган Федеральной службы госуд...,79,Саратовская область,"Региональный, без структуры"
43738,1054521.38,NaN,3881593,2020,Савельев Сергей Вячеславович,NaN,NaN,супруг,NaN,Территориальный орган Федеральной службы госуд...,79,Саратовская область,"Региональный, без структуры"
43742,934695.80,NaN,3881600,2020,Ватман Александр Михайлович,NaN,NaN,супруг,NaN,Территориальный орган Федеральной службы госуд...,79,Саратовская область,"Региональный, без структуры"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131368,281643.10,NaN,3792161,2020,Аверкиев Дмитрий Алексеевич,NaN,NaN,супруг,NaN,Павинский район,53,Костромская область,"Муниципальный, без структуры"
131370,515813.69,NaN,3792178,2020,Кузнецов Сергей Валентинович,NaN,NaN,супруг,NaN,Павинский район,53,Костромская область,"Муниципальный, без структуры"
131372,373081.46,NaN,3792185,2020,Смирнов Николай Владимирович,NaN,NaN,супруг,NaN,Павинский район,53,Костромская область,"Муниципальный, без структуры"
131374,711446.81,NaN,3792236,2020,Шишов Сергей Владимирович,NaN,NaN,супруг,NaN,Павинский район,53,Костромская область,"Муниципальный, без структуры"


In [6]:
# смотрим, есть ли в столбце department подобные пометки
draft[(pd.notna(draft['region_name'])) & ((draft['department']=='супруг') | (draft['department']=='супруга') | (draft['department']=='дочь') | (draft['department']=='сын'))]

,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name


In [7]:
# дропнем лишние данные
df = draft.drop(draft[((draft['position']=='супруг') | (draft['position']=='супруга') | (draft['position']=='дочь') | (draft['position']=='сын'))].index).reset_index()
len(df)

131747

## Проставляем регион по шагам (названия столбцов + действия)

Регион будем проставлять в отдельном столбце region. Начнем с очевидного - перенесем все известные регионы, далее будем просматривать в ручном режиме либо использовать список cities&regions, где собраны названия городов региона и другие ключевые слова для поиска. (например, только прилагательное из названия региона (Амурской области - Амурский) и т.д.). 

Принципы формирования списка cities&regions:
 - взят список городов с регионами с сайта https://hramy.ru/regions/city_reg.htm
 - выборочно проверила, что за города, косяков не нашла.
 - названия регионов исправлены так, чтобы они совпадали с названиями в Деклараторе.
 - в конце списка добавлены вариации на названия региона без окончания (например, Мордовск - Республика Мордовия), если название региона не происходит напрямую от названия города, находящегося в этом регионе (например, добавляем Амурск для Амурской области, т.к. в Амурской области нет города Амура / Амурска; а Новосибирский для НСО не добавляю, т.к. город Новосибирск уже есть в списке и с точки зрения написания это слово является частью слова Новосибирской)
 - удалены названия городов, которые могут запутать поиск (так в Хабаровском крае есть город Амурск, если искать по нему, то все чиновники с должностями, относящимися к Амурской области, будут отнесены к Хабаровскому краю), если такое удаление не повлечет за собой серьезных искажений.
 - удалены города с одинаковыми названиями в разных регионах (кроме Благовещенска, оставила для Амурской области).
 - добавлены точные названия/вариации названий учреждений (например, Алтайский ЦСМ - Алтайский край).
 
**Примечания:** 
1) если учреждение занимается задачи в нескольких регионах (например, правления Роскомнадзора по Тюменской области, Ханты-Мансийскому автономному округу-Югре и Ямало-Ненецкому автономному округу), то относится на первый по алфавиту регион.  
2) pymystem3 и подобные библиотеки не использовать, т.к. автоматический поиск не сможет определить Уфа - это город или часть какого-то большого слова, т.к. при лемматизации предложения превращаются в набор слов в именнительном падеже мужского рода без заглавных букв.
 

Пойдем от больших групп к разрозненным элементам: 
office_name -> position -> department

#### Шаг 1. name: 
тут все очевидно, с помощью формулы проставляем название региона в столбец region

#### Шаг 2. office_name:
 - выделим регион, там где очевидно (например, по названию)
 - для остальных воспользуемся списком cities&regions
 
#### Шаг 3. position:
 - воспользуемся только списком cities&regions, т.е. позиций много, просматривать глазами трудозатратно
 
#### Шаг 4. department:
 - воспользуемся только списком cities&regions, т.е. позиций много, просматривать глазами трудозатратно
 
На каждом шаге будем отмечать, изменение заполненных строк в столбце region.
А также создавать отдельный датафрейм с номером шага (df1, df2, etc.)

In [8]:
# строки, где нет региона, перед началом пошаговой обработки
# False - нет региона, True - есть регион
pd.notna(df['region_name']).value_counts()

True     107375
False     24372
Name: region_name, dtype: int64

### Шаг 1. name
C помощью формулы проставляем название региона в столбец region.

In [9]:
def region_1(row):
    # если у чиновника есть регион, то выносим его в столбец region
    if pd.notna(row['region_name']):
        return row['region_name']

In [10]:
# создадим копию предыдущего датафрейма, чтобы было проще "откатить" этап назад
df1 = df.copy()

In [11]:
# создадим столбец region и построчно перенесем в него название региона
df1['region'] = df1.apply(region_1, axis=1)
df1

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region
0,0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой,Москва
1,1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой,Москва
2,2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
3,3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
4,4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131742,131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131743,131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131744,131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131745,131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область


In [12]:
# проверим, как заполнились строки в region (должны совпадать с аналогичным расчетом для датафрейма draft)
# False - нет региона, True - есть регион
pd.notna(df1['region']).value_counts()

True     107375
False     24372
Name: region, dtype: int64

### Шаг 2. office_name
 - выделим регион, там где очевидно (например, по названию)
 - для остальных воспользуемся списком cities&regions
 
Столбец name содержит верхнеуровневую классификацию мест работы чиновников (федеральный/региональный, со структурой/без структуры). Название конкретного министерства/ведомства/учреждения ищем в столбце office_name.  

Если office_name позволяет однозначно отнести его к тому или иному региону, то добавляем его в cities&regions, который будем использовать, чтобы столбец office_name на принадлежность к тому или иному региону.

In [13]:
# для каких типов офиса регион не определен
df1_nan = df1.loc[pd.isna(df1['region'])]

df1_nan.pivot_table(index=['name'], values='size', aggfunc='count', margins=True).astype(int)

,size
name,
"Надрегиональный, без структуры",3474
"Организация, c региональной структурой",281
"Организация, без структуры",5
"Федеральный, без региональной структуры",8058
"Федеральный, с региональной структурой",11576
"Федеральный, со структурой",978
All,24372


##### 1) Надрегиональный, без структуры

In [31]:
# рассмотрим каждый office_type в отдельности
# поймем есть ли возможность определить регионы для отдельных групп
df1_nan[df1_nan['name']=='Надрегиональный, без структуры'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

,size
office_name,
07-й арбитражный апелляционный суд (г. Томск),1
13-й арбитражный апелляционный суд (г. Санкт-Петербург),6
17-й арбитражный апелляционный суд (г. Пермь),18
20-й арбитражный апелляционный суд (г. Тула),30
Court for intellectual property rights,66
...,...
ЯРБ Донское межрегиональное территориальное управление по надзору за ядерной и радиационной безопасностью,17
ЯРБ Межрегиональное территориальное управление по надзору за ядерной и радиационной безопасностью Сибири и Дальнего Востока,16
ЯРБ Северо-Европейское межрегиональное территориальное управление по надзору за ядерной и радиационной безопасностью,7


In [27]:
# сохраним в эксель и просмотрим глазами, какие office_name попали в тип "Надрегиональный, без структуры"
df1_nan[df1_nan['name']=='Надрегиональный, без структуры'].pivot_table(index=['name', 'office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int).to_excel('надрегиональный.xlsx')

# следующие позиции добавила в список cities&regions:
# Court for intellectual property rights (Москва),
# Военная прокуратура Балтийского флота (Калининградская область),
# Московская городская военная прокуратура (Москва).

##### 2) Организация, c региональной структурой

In [32]:
df1_nan[df1_nan['name']=='Организация, c региональной структурой'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

# будем делить по регионам на шаге 3 

,size
office_name,
Пенсионный фонд Российской Федерации,223
Фонд социального страхования,58
All,281


##### 3) Организация, без структуры

In [33]:
df1_nan[df1_nan['name']=='Организация, без структуры'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

# Байконур - за рамками географии РФ, не интересует

,size
office_name,
Отделение Пенсионного фонда Российской Федерации г. Байконур,5
All,5


##### 4) Федеральный, без региональной структуры

In [34]:
df1_nan[df1_nan['name']=='Федеральный, без региональной структуры'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

# внесла в cities&regions Accounts chamber, Администрация Президента РФ, Аппарат Государственной Думы и др.,
# которые можно однозначно отнести к Москве

,size
office_name,
Accounts chamber,910
Администрация Президента РФ,80
Аппарат Государственной Думы,649
Аппарат Правительства РФ,73
Аппарат Совета Федерации,133
Генеральная прокуратура,175
Главное управление специальных программ Президента,3
Государственная корпорация по космической деятельности «Роскосмос»,163
Министерство здравоохранения,324


##### 5) Федеральный, с региональной структурой

In [53]:
df1_nan[df1_nan['name']=='Федеральный, с региональной структурой'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

# у всех приведенных служб/ведомств есть региональные представительства, которые рассмотрим на шаге 3

,size
office_name,
Главная военная прокуратура,33
Министерство внутренних дел,44
"Министерство по делам гражданской обороны, чрезвычайным ситуациям и ликвидации последствий стихийных бедствий",173
Министерство юстиции,340
Федеральная антимонопольная служба,208
Федеральная налоговая служба,125
Федеральная служба в сфере защиты прав потребителей и благополучия человека,401
"Федеральная служба государственной регистрации, кадастра и картографии",651
Федеральная служба государственной статистики,359


##### 6) Федеральный, со структурой

In [55]:
df1_nan[df1_nan['name']=='Федеральный, со структурой'].pivot_table(index=['office_name'],
                                 values='size', aggfunc='count', margins=True).astype(int)

# Государственная Дума, Совет Федерации, Совет Федерации относим к Москве, 
# у остальных служб и агентств есть региональная структура, поэтому их рассмотрим на шаге 3

,size
office_name,
Государственная Дума,443
Правительство РФ,32
Совет Федерации,166
Федеральная служба по техническому и экспортному контролю,89
Федеральное агентство воздушного транспорта,101
Федеральное агентство по государственным резервам,147
All,978


In [13]:
# создадим копию предыдущего датафрейма, чтобы было проще "откатить" этап назад
df2 = df1.copy()

In [14]:
# список регионов cities&regions сначала помещаем в датафрейм
cities_regions = pd.read_excel('cities&regions.xlsx', sheet_name='list').set_index('city')
#cities_regions
# а потом превращаем в словарь
regions_dict = cities_regions.to_dict()['region_Declarator']
regions_dict

{'Санкт-Петербург': 'Санкт-Петербург',
 'Федеральное государственное бюджетное учреждение "Управление эксплуатации Большого Ставропольского канала"': 'Карачаево-Черкесская республика',
 'Адыгейск': 'Республика Адыгея',
 'Майкоп': 'Республика Адыгея',
 'Агидель': 'Республика Башкортостан',
 'Баймак': 'Республика Башкортостан',
 'Белебей': 'Республика Башкортостан',
 'Белорецк': 'Республика Башкортостан',
 'Давлеканово': 'Республика Башкортостан',
 'Дюртюли': 'Республика Башкортостан',
 'Ишимбай': 'Республика Башкортостан',
 'Кумертау': 'Республика Башкортостан',
 'Межгорье': 'Республика Башкортостан',
 'Мелеуз': 'Республика Башкортостан',
 'Нефтекамск': 'Республика Башкортостан',
 'Салават': 'Республика Башкортостан',
 'Сибай': 'Республика Башкортостан',
 'Стерлитамак': 'Республика Башкортостан',
 'Туймазы': 'Республика Башкортостан',
 'Уфа': 'Республика Башкортостан',
 'Учалы': 'Республика Башкортостан',
 'Янаул': 'Республика Башкортостан',
 'Закаменск': 'Республика Бурятия',
 'Кяхта':

In [15]:
# функция, которая построчно проставляет регион
def region_2(row):
    # проверяем, заполнен ли столбец region
    if pd.notna(row['region']):
        # и оставляем все как есть
        return row['region']
    # если не заполнен (NaN)
    else:
        # проверим, есть ли названия городов или учреждений в списке cities&regions
        for key, value in regions_dict.items():
                if key in row['office_name']:
                    return value

In [16]:
df2['region'] = df2.apply(region_2, axis=1)
df2

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region
0,0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой,Москва
1,1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой,Москва
2,2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
3,3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
4,4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131742,131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131743,131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131744,131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131745,131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область


In [17]:
# проверим, как заполнились строки в region
# False - нет региона, True - есть регион
pd.notna(df2['region']).value_counts()

True     110944
False     20803
Name: region, dtype: int64

### Шаг 3. position
Будем искать в столбце position название города из словаря regions_dict (cities&regions).

In [18]:
# создадим копию предыдущего датафрейма, чтобы было проще "откатить" этап назад
df3 = df2.copy()

In [19]:
def region_3(row):
    # проверяем, заполнен ли столбец region
    if pd.notna(row['region']):
        # и оставляем все как есть
        return row['region']
    
    # если не заполнен
    else:
        # в столбце position есть значения NaN
        # чтобы избежать проблем при обработки таких полей, напишем отдельное условие
        if pd.isna(row['position']):
            return None
        
        else:
            for key, value in regions_dict.items():
                if key in row['position']:
                    return value

In [20]:
df3['region'] = df3.apply(region_3, axis=1)
df3

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region
0,0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой,Москва
1,1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой,Москва
2,2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
3,3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
4,4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131742,131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131743,131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131744,131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131745,131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область


In [21]:
# проверим, как заполнились строки в region
# False - нет региона, True - есть регион
pd.notna(df3['region']).value_counts()

True     112993
False     18754
Name: region, dtype: int64

### Шаг 4. department
Все действия аналогичны шагу 3.

In [22]:
# создадим копию предыдущего датафрейма, чтобы было проще "откатить" этап назад
df4 = df3.copy()

In [23]:
def region_4(row):
    # проверяем, заполнен ли столбец region
    if pd.notna(row['region']):
        # и оставляем все как есть
        return row['region']
    
    # если не заполнен
    else:
        # в столбце department есть значения NaN
        # чтобы избежать проблем при обработки таких полей, напишем отдельное условие
        if pd.isna(row['department']):
            return None
        
        else:
            for key, value in regions_dict.items():
                if key in row['department']:
                    return value

In [24]:
df4['region'] = df4.apply(region_4, axis=1)
df4

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region
0,0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой,Москва
1,1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой,Москва
2,2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
3,3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
4,4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131742,131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131743,131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131744,131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область
131745,131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область


In [25]:
# проверим, как заполнились строки в region
# False - нет региона, True - есть регион
pd.notna(df4['region']).value_counts()

True     120017
False     11730
Name: region, dtype: int64

# Проверки

### 1) Проверка на количество регионов

In [26]:
# должно быть 85
# и количество деклараций по регионам, там, где мало, поискать есть ли возможность доп.поиска региона
pivot = df4[pd.notna(df4['region'])].pivot_table(index='region', values='size', 
                                         aggfunc=['count', 'mean']).astype(int)
pivot

,count,mean
,size,size
region,,
Алтайский край,2538,1414252
Амурская область,1544,1554603
Архангельская область,1654,1490239
Астраханская область,1008,1193924
Белгородская область,1805,1314044
...,...,...
Чеченская республика,533,6901446
Чувашская республика - Чувашия,385,1912856


In [30]:
#pivot.to_excel('pivot.xlsx')

### 2) Проверка на корретность отнесения по регионам

Потенциально возможна путаница с отнесением к таким регионам как:
 - Москва и Московская область,
 - Алтайский край и Республика Алтай,
 - Ненецкий АО и Ямало-Ненецкий АО
 
Проверяю вручную также и по другим регонам, где есть подозрение, что возникла путаница.

In [31]:
x = df4.loc[pd.isna(df4['region_name']) & pd.notna(df4['region']) & pd.notna(df4['position'])]
y = x.loc[x['position'].str.contains('Ставрополь')].pivot_table(index=['position', 'region'])
y

,,income_year,index,section_id,size
position,region,,,,
Администратор Арбитражного суда Ставропольского края,Ставропольский край,2020,61007.0,3901293.0,6.797292e+05
Администратор Ставропольского гарнизонного военного суда,Ставропольский край,2020,61009.0,3901295.0,5.209757e+05
Врио начальника Управления Федеральной службы войск национальной гвардии Российской Федерации по Ставропольскому краю,Ставропольский край,2020,34749.0,1624924.0,1.604734e+06
"Генеральный директор ФБУ ""Ставропольский ЦСМ""",Ставропольский край,2020,37594.0,1574882.0,3.851062e+06
Главный врач ФБУЗ «Центр гигиены и эпидемиологии в Ставропольском крае»,Ставропольский край,2020,58674.0,1497012.0,3.853991e+06
"Директор ФГУП ""Рассвет-Ставрополье""",Ставропольский край,2020,56762.0,1625066.0,7.820485e+05
Директор ФКУЗ «Ставропольский научно-исследовательский противочумный институт»,Ставропольский край,2020,58717.0,1497055.0,6.141303e+06
Директор филиала ЦЛАТИ по Ставропольскому краю,Ставропольский край,2020,35776.0,1572289.0,2.312924e+06
Заместитель начальника ГУ Минюста России по Ставропольскому краю,Ставропольский край,2020,34364.0,1593115.0,1.182606e+06


In [32]:
x = df4.loc[pd.isna(df4['region_name']) & pd.notna(df4['region']) & pd.notna(df4['department'])]
y = x.loc[x['department'].str.contains('Ставрополь')].pivot_table(index=['department', 'region'])
y

,,income_year,index,section_id,size
department,region,,,,
29. Филиал ФГБУ «ФКП Росреестра» по Ставропольскому краю,Ставропольский край,2020,10618.5,1502591.5,1.317997e+06
Ставропольский отдел государственного экологического надзора,Карачаево-Черкесская республика,2020,58954.5,3898745.5,6.701120e+05
УФСИН России по Ставропольскому краю,Ставропольский край,2020,46194.5,1558585.5,1.927882e+06
ФКУ БМТиВС УФСИН России по Ставропольскому краю,Ставропольский край,2020,46260.5,1558651.5,1.074441e+06
ФКУ ИК-1 УФСИН России по Ставропольскому краю,Ставропольский край,2020,46199.5,1558590.5,1.235781e+06
ФКУ ИК-11 УФСИН России по Ставропольскому краю,Ставропольский край,2020,46240.5,1558631.5,1.255965e+06
ФКУ ИК-2 УФСИН России по Ставропольскому краю,Ставропольский край,2020,46204.5,1558595.5,9.128073e+05
ФКУ ИК-3 УФСИН России по Ставропольскому краю,Ставропольский край,2020,46210.0,1558601.0,9.054468e+05
ФКУ ИК-4 УФСИН России по Ставропольскому краю,Ставропольский край,2020,46215.5,1558606.5,1.048103e+06


### 3) Проверка на дубликаты деклараций за конкретный год

Сначала создадим один столбец для имени чиновника - person_name, куда подтянем значения из original_fio	и full_name.

In [27]:
def name(row):
    # декларации без региона нас не интересуют
    if pd.isna(row['region']):
        return None
    
    else:
        # если поле full_name заполнено, берем его
        if pd.notna(row['full_name']):
            return row['full_name']
        
        else:
            # иначе - original_fio
            return row['original_fio']

In [28]:
DFcheck = df4.copy()

DFcheck['person_name'] = DFcheck.apply(name, axis=1)
DFcheck

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
0,0,14154017.82,NaN,774180,2020,NaN,Алексей Анатольевич Пашков,8001,Префект Северо-Западного административного окр...,NaN,Префектура Северо-Западного административного ...,63,Москва,Региональный со структурой,Москва,Алексей Анатольевич Пашков
1,1,13936977.62,NaN,774181,2020,NaN,Дмитрий Владимирович Набокин,3447,Префект Троицкого и Новомосковского администра...,NaN,Префектура Троицкого и Новомосковского округов,63,Москва,Региональный со структурой,Москва,Дмитрий Владимирович Набокин
2,2,13863391.25,NaN,774182,2020,NaN,Олег Александрович Волков,3451,Префект Юго-Западного административного округа...,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва,Олег Александрович Волков
3,3,6042983.19,NaN,3466993,2020,Ломова Елена Николаевна,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва,Ломова Елена Николаевна
4,4,4497564.40,NaN,3466995,2020,Алисултанов Анзор Рашидович,NaN,NaN,первый заместитель префекта,NaN,Префектура Юго-Западного административного округа,63,Москва,Региональный со структурой,Москва,Алисултанов Анзор Рашидович
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131742,131871,394478.13,NaN,3796731,2020,Филиппова И.Б.,NaN,NaN,Главный специалист отдела общего образования и...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область,Филиппова И.Б.
131743,131872,208731.59,NaN,3796733,2020,Шумкова В.М.,NaN,NaN,Специалист 1 категории отдела опеки и попечите...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область,Шумкова В.М.
131744,131873,564185.86,NaN,3796737,2020,Чувозерова Н.А.,NaN,NaN,Ведущий специалист отдела опеки и попечительства,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область,Чувозерова Н.А.
131745,131874,386499.31,NaN,3796741,2020,Язовских Н.В.,NaN,NaN,Специалист 1 категории сектора по обеспечению ...,NaN,Курган - городской округ,98,Курганская область,"Муниципальный, без структуры",Курганская область,Язовских Н.В.


Используем pandas.DataFrame.duplicated - выводит дубли (mark duplicates as True except for the first occurrence).

Исключаем дубли в 2 шага:
1) сначала убираем строки, где совпадают size, person_name, region - это просто дублирующиеся декларации (например, потому что чиновник одновременно является ректором универа и депутатом и сдает декларации в этих двух качествах).  

2) отдельно рассматривает случаи, когда у чиновника совпадают только size и person_name, а регион отличается, значит имели место кадровые перестановки и нужно "вручную" убедится, в каком регионе чиновник проработал большую часть года.

3) совпадает size, original_fio, region - ситуация аналогична 1).

4) совпадает size, region - это случаи, когда имя чиновника внесено по-разному (например, порядок в ФИО не совпадает).  

Случаи, когда совпадает только size не рассматриваю в целях экономии времени.

##### 1) size, person_name, region совпадают

In [29]:
# выбираем только строки, где проставлен регион
df2020 = DFcheck.loc[pd.notna(DFcheck['region'])]

# определяем дубликаты по сумме, имени и региону
# вносим индексы дублей в список для удобства последующего исключения
indexes_excl2020 = df2020[df2020[['size', 'person_name', 'region']].duplicated()].index.to_list()
len(indexes_excl2020)

2424

In [30]:
# подробнее посмотрим, что это за дубли
df2020[df2020[['size', 'person_name', 'region']].duplicated()]

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
2370,2370,6570665.16,NaN,776727,2020,NaN,Инна Юрьевна Святенко,2589,Депутат Московской городской думы,NaN,Московская городская Дума,63,Москва,Региональный со структурой,Москва,Инна Юрьевна Святенко
2608,2608,3153344.53,NaN,3727463,2020,Борисенко Игорь Владимирович,NaN,NaN,исполняющий обязанности начальника Департамент...,NaN,Правительство Смоленской области,100,Смоленская область,Региональный со структурой,Смоленская область,Борисенко Игорь Владимирович
2788,2788,2022549.00,NaN,1466146,2020,Алёхин В.В.,Валерий Владимирович Алёхин,13556,Депутат государственного собрания Республики М...,NaN,Государственное собрание Республики Мордовия,62,Республика Мордовия,Региональный со структурой,Республика Мордовия,Валерий Владимирович Алёхин
2792,2792,2446525.00,NaN,1466150,2020,Аширов Р.З.,Рафаиль Закиевич Аширов,13557,Депутат государственного собрания Республики М...,NaN,Государственное собрание Республики Мордовия,62,Республика Мордовия,Региональный со структурой,Республика Мордовия,Рафаиль Закиевич Аширов
2806,2806,1397817.00,NaN,1466164,2020,Киселева С.Е.,Светлана Евгеньевна Киселева,138146,Депутат государственного собрания Республики М...,NaN,Государственное собрание Республики Мордовия,62,Республика Мордовия,Региональный со структурой,Республика Мордовия,Светлана Евгеньевна Киселева
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131146,131252,456907.07,NaN,3791614,2020,Исаченко Вера Валерьевна,NaN,NaN,Глава городского поселения город Макарьев на н...,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",Костромская область,Исаченко Вера Валерьевна
131157,131263,692028.81,NaN,3791745,2020,Зудов Алексей Игоревич,NaN,NaN,Депутат Совета депутатов Унженского сельского ...,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",Костромская область,Зудов Алексей Игоревич
131167,131273,371920.16,NaN,3791841,2020,Соколов Максим Владимирович,NaN,NaN,Депутат Совета депутатов городского поселения ...,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",Костромская область,Соколов Максим Владимирович
131168,131274,456907.07,NaN,3791853,2020,Исаченко Вера Валерьевна,NaN,NaN,Глава городского поселения город Макарьев на н...,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",Костромская область,Исаченко Вера Валерьевна


In [31]:
# убираем дубли из итогового датафрейма
DFcheck.loc[DFcheck.index.isin(indexes_excl2020), 'region'] = None

In [32]:
# проверяем все ли дубли исключены

a = len(df4[(pd.notna(df4['region'])) & (df4['income_year']==2020)])
b = len(DFcheck[(pd.notna(DFcheck['region'])) & (DFcheck['income_year']==2020)])

print(f'Количество деклараций с определенным регионом в 2020 (со всеми дублями): ', a)
print(f'Количество деклараций с определенным регионом в 2020 (без дублей): ', b)
print(f'Разница: ', a-b)

if a-b == len(indexes_excl2020):
    print(f'Дубли исключены корректно')

Количество деклараций с определенным регионом в 2020 (со всеми дублями):  120017
Количество деклараций с определенным регионом в 2020 (без дублей):  117593
Разница:  2424
Дубли исключены корректно


In [33]:
# выборочно посмотрим, что по дублям стоит None в столбце region
DFcheck[DFcheck['person_name']=='Макарушина Елена Александровна']

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
131144,131250,441668.98,NaN,3791171,2020,Макарушина Елена Александровна,NaN,NaN,Директор МКУ «ЕДДиОС»,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",Костромская область,Макарушина Елена Александровна
131169,131275,441668.98,NaN,3791858,2020,Макарушина Елена Александровна,NaN,NaN,Депутат Совета депутатов городского поселения ...,NaN,Макарьевский район,53,Костромская область,"Муниципальный, без структуры",None,Макарушина Елена Александровна


##### 2) size, person_name совпадают

In [34]:
# теперь рассмотрим случаи, где имя и сумма совпадают, а регион - нет
# заново делаем выборку, т.к. базовый датафрейм DFcheck был изменен на предыдущей итерации
df2020 = DFcheck.loc[pd.notna(DFcheck['region'])]

df2020[df2020[['size', 'person_name']].duplicated(keep=False)]

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
1487,1487,16068823.00,NaN,776199,2020,NaN,Сергей Иванович Меняйло,31754,Полномочный представитель Президента РФ в феде...,NaN,Администрация Президента РФ,NaN,NaN,"Федеральный, без региональной структуры",Москва,Сергей Иванович Меняйло
1685,1685,7165610.46,NaN,776781,2020,NaN,Алексей Юрьевич Русских,53,Член Совета Федерации от Московской области,NaN,Совет Федерации,NaN,NaN,"Федеральный, со структурой",Москва,Алексей Юрьевич Русских
3233,3233,7165610.46,NaN,777298,2020,NaN,Алексей Юрьевич Русских,53,Временно исполняющий обязанности губернатора У...,NaN,Губернатор Ульяновской области,88,Ульяновская область,"Региональный, без структуры",Ульяновская область,Алексей Юрьевич Русских
3887,3887,4071937.24,NaN,3729875,2020,Еремин А.Л.,NaN,NaN,Первый заместитель руководителя управления,NaN,ГСУ СК РФ Красноярский край и Республика Хакасия,28,Красноярский край,"Федеральный, без региональной структуры",Красноярский край,Еремин А.Л.
3888,3888,6404738.68,NaN,3729881,2020,Зюбанов А.Г.,NaN,NaN,Заместитель руководителя Главного управления,NaN,ГСУ СК РФ Красноярский край и Республика Хакасия,28,Красноярский край,"Федеральный, без региональной структуры",Красноярский край,Зюбанов А.Г.
3889,3889,5668757.40,NaN,3729894,2020,Курочкин П.В.,NaN,NaN,Заместитель руководителя Главного управления,NaN,ГСУ СК РФ Красноярский край и Республика Хакасия,28,Красноярский край,"Федеральный, без региональной структуры",Красноярский край,Курочкин П.В.
3890,3890,4505623.82,NaN,3729898,2020,Николайчук Е.М.,NaN,NaN,Заместитель руководителя Главного управления,NaN,ГСУ СК РФ Красноярский край и Республика Хакасия,28,Красноярский край,"Федеральный, без региональной структуры",Красноярский край,Николайчук Е.М.
3891,3891,5413450.32,NaN,3729902,2020,Позовский Д.А.,NaN,NaN,Заместитель руководителя Главного управления,NaN,ГСУ СК РФ Красноярский край и Республика Хакасия,28,Красноярский край,"Федеральный, без региональной структуры",Красноярский край,Позовский Д.А.
8688,8688,1774479.93,NaN,1650443,2020,Шариков А.С.,NaN,NaN,Прокурор Курманаевского района,NaN,Прокуратура Оренбургской области,105,Оренбургская область,"Региональный, без структуры",Оренбургская область,Шариков А.С.
8803,8803,4071937.24,NaN,3755416,2020,Еремин А.Л.,NaN,NaN,Первый заместитель руководителя управления,NaN,СУ СК РФ Тульская область,84,Тульская область,"Федеральный, без региональной структуры",Тульская область,Еремин А.Л.


In [41]:
# выгрузим в эксель и проверим глазами каждого
# df2020[df2020[['size', 'person_name']].duplicated(keep=False)].to_excel('2.xlsx')

Чиновники, которые поменяли должности:

 - Меняйло Сергей Иванович: до апреля 2021 - полпред в СФО (индекс 1487), после - врио главы Сев.Осетии (67807) - не относится к 2020г., убираем эту строку.  
 - Русских Алексей Юрьевич: до апреля 2021 - член СФ (1685), после - врио губера Ульяновской области (3233) - не относится к 2020г., убираем эту строку.
 - Шариков А.С.: до апреля 2021 - прокурор в Оренбургской области (8688), после - прокурор в Самарской области (22305) - не относится к 2020г., убираем эту строку.
 - Кононов Эдуард Николаевич: с октября 2020 - в Тульской области (18418), до октября 2020 - в Орловской области (11592) - уберу последнюю, т.к. на конец 2020 был в новом регионе.
 
Чиновники, по которым некорректно определился регион:
 - Аксютина Татьяна Викторовна: некорректный регион - позиция 18417.
 - Еремин А.Л.: некорректный регион - позиция 8803.
 - Жданов Андрей Евгеньевич: некорректный регион - позиция 18416.
 - Зюбанов А.Г.: некорректный регион - позиция 8804.
 - Курочкин П.В.: некорректный регион - позиция 8805.
 - Николайчук Е.М.: некорректный регион - позиция 8806.
 - Ногаев Алан Юрьевич: некорректный регион - позиция 18419.
 - Позовский Д.А.: некорректный регион - позиция 8807.
 - Смирнов Михаил Витальевич: некорректный регион - позиция 18415.

In [35]:
# вносим индексы в список
indexes_excl2020_2 = [67807, 3233, 22305, 11592, 18417, 8803, 18416, 8804, 8805, 8806, 18419, 8807, 18415]

# убираем для них регион
DFcheck.loc[DFcheck.index.isin(indexes_excl2020_2), 'region'] = None

In [36]:
# выборочно посмотрим, что по дублям стоит None в столбце region
DFcheck[DFcheck['person_name']=='Смирнов Михаил Витальевич']

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
11589,11589,2202187.86,NaN,3767797,2020,Смирнов Михаил Витальевич,NaN,NaN,Руководитель УФССП России по Орловской области...,NaN,Управление Федеральной службы судебных пристав...,70,Орловская область,"Региональный, без структуры",Орловская область,Смирнов Михаил Витальевич
18415,18415,2202187.86,NaN,3796564,2020,Смирнов Михаил Витальевич,NaN,NaN,Руководитель УФССП России по Орловской области...,NaN,Управление Федеральной службы судебных пристав...,84,Тульская область,"Региональный, без структуры",None,Смирнов Михаил Витальевич


##### 3) size, original_fio, region совпадает

In [37]:
# заметила, что есть позиции, где сопадает сумма и original_fio, отфильтруем по этому признаку
df2020 = DFcheck.loc[pd.notna(DFcheck['region'])]

df2020[df2020[['size', 'original_fio', 'region']].duplicated(keep=False)]

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
1838,1838,5611777.85,NaN,776937,2020,NaN,Борис Михайлович Гладких,15065,"Депутат государственной думы, член комитета",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Борис Михайлович Гладких
1901,1901,5611082.40,NaN,777000,2020,NaN,Рамзил Рафаилович Ишсарин,31987,"Депутат государственной думы, член комитета",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Рамзил Рафаилович Ишсарин
1962,1962,5611777.85,NaN,777061,2020,NaN,Юрий Андреевич Левицкий,12626,"Депутат государственной думы, член комитета",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Юрий Андреевич Левицкий
1988,1988,5611082.40,NaN,777087,2020,NaN,Иршат Султанович Минкин,31981,"Депутат государственной думы, член комитета",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Иршат Султанович Минкин
2026,2026,5611777.85,NaN,777125,2020,NaN,Денис Андреевич Парфенов,31952,"Депутат государственной думы, член комитета",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Денис Андреевич Парфенов
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56938,56944,8215955.42,NaN,1625248,2020,Федорук М.П.,Михаил Петрович Федорук,21568,"ректор, ФГАОУ ВО ""Новосибирский национальный и...",NaN,Министерство науки и высшего образования,NaN,NaN,"Федеральный, без региональной структуры",Новосибирская область,Михаил Петрович Федорук
58268,58274,1374281.43,NaN,1602223,2020,Беляков Н.В.,Николай Владимирович Беляков,173495,Руководитель Управления Роскомнадзора по Томск...,NaN,"Министерство цифрового развития, связи и массо...",NaN,NaN,"Федеральный, без региональной структуры",Томская область,Николай Владимирович Беляков
58316,58322,1159925.03,NaN,1602271,2020,Секнин А.Л.,Александр Леонтьевич Секнин,62192,Руководитель Управления Роскомнадзора по Респу...,NaN,"Министерство цифрового развития, связи и массо...",NaN,NaN,"Федеральный, без региональной структуры",Республика Башкортостан,Александр Леонтьевич Секнин
68187,68193,1605763.05,NaN,3907994,2020,Анпилогов А.В.,NaN,NaN,Руководитель Управления,NaN,Управление Роскомнадзора Курская область,55,Курская область,"Региональный, без структуры",Курская область,Анпилогов А.В.


In [38]:
indexes_excl2020_3 = df2020[df2020[['size', 'original_fio', 'region']].duplicated()].index.to_list()
len(indexes_excl2020_3)

245

In [39]:
# убираем дубли из итогового датафрейма
DFcheck.loc[DFcheck.index.isin(indexes_excl2020_3), 'region'] = None

In [40]:
# выборочно посмотрим, что по дублям стоит None в столбце region
DFcheck[DFcheck['original_fio']=='Анпилогов А.В.']

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
35878,35878,1605763.05,NaN,1507030,2020,Анпилогов А.В.,Андрей Викторович Анпилогов,62136,Руководитель Управления Роскомнадзора по Курск...,NaN,"Федеральная служба по надзору в сфере связи, и...",NaN,NaN,"Федеральный, с региональной структурой",Курская область,Андрей Викторович Анпилогов
58265,58271,1605763.05,NaN,1602220,2020,Анпилогов А.В.,Андрей Викторович Анпилогов,62136,Руководитель Управления Роскомнадзора по Курск...,NaN,"Министерство цифрового развития, связи и массо...",NaN,NaN,"Федеральный, без региональной структуры",None,Андрей Викторович Анпилогов
68187,68193,1605763.05,NaN,3907994,2020,Анпилогов А.В.,NaN,NaN,Руководитель Управления,NaN,Управление Роскомнадзора Курская область,55,Курская область,"Региональный, без структуры",None,Анпилогов А.В.


##### 4) совпадает size и region

In [41]:
# заново делаем выборку, т.к. базовый датафрейм DFcheck был изменен на предыдущей итерации
df2020 = DFcheck.loc[pd.notna(DFcheck['region'])]

# возьмем только повторы суммарно по каждому региону (количество и общая сумма)
# напоминалка: .duplicated() берет только повтор значения (Mark duplicates as True except for the first occurrence)
df2020[df2020[['size', 'region']].duplicated()]

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
382,382,5145340.42,NaN,3726681,2020,Цыденов А.С.,NaN,NaN,Глава Республики Бурятия – Председатель Правит...,NaN,Глава республики Бурятия,5,Республика Бурятия,"Региональный, без структуры",Республика Бурятия,Цыденов А.С.
2097,2097,5612436.55,NaN,777196,2020,СЛУЦКИЙ Леонид Эдуардович,Леонид Эдуардович Слуцкий,105,"Депутат государственной думы, председатель ком...",NaN,Государственная Дума,NaN,NaN,"Федеральный, со структурой",Москва,Леонид Эдуардович Слуцкий
2708,2708,103200.00,NaN,1627857,2020,Скорых Олег Николаевич,Олег Николаевич Скорых,203050,депутат на непостоянной профессиональной основе,NaN,Народное Собрание Карачаево-Черкесской республики,11,Карачаево-Черкесская республика,Региональный со структурой,Карачаево-Черкесская республика,Олег Николаевич Скорых
3104,3104,1255706.72,в том числе пособие по временной нетрудоспособ...,777297,2020,NaN,Бату Сергеевич Хасиков,4928,Глава Республики Калмыкия,NaN,Глава республики Калмыкия,47,Республика Калмыкия,"Региональный, без структуры",Республика Калмыкия,Бату Сергеевич Хасиков
4354,4354,25280265.47,NaN,1463793,2020,NaN,Дмитрий Андреевич Артюхов,22552,Губернатор ЯНАО,NaN,Губернатор Ямало-Ненецкого АО,104,Ямало-Ненецкий автономный округ,"Региональный, без структуры",Ямало-Ненецкий автономный округ,Дмитрий Андреевич Артюхов
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130542,130648,811988.91,NaN,3789900,2020,2. ПОРОШКИН Андрей Иванович,NaN,NaN,Начальник управления культуры,NaN,Сыктывдинский район,13,Республика Коми,"Муниципальный, без структуры",Республика Коми,2. ПОРОШКИН Андрей Иванович
130548,130654,694154.68,NaN,3789930,2020,Муравьев Владимир Николаевич,NaN,NaN,директор,NaN,Сыктывдинский район,13,Республика Коми,"Муниципальный, без структуры",Республика Коми,Муравьев Владимир Николаевич
130622,130728,0.00,NaN,3781809,2020,Павлова Арина Сергеевна,NaN,NaN,NaN,NaN,Троицко-Печорский район,13,Республика Коми,"Муниципальный, без структуры",Республика Коми,Павлова Арина Сергеевна
130649,130755,1191041.15,NaN,3782029,2020,12. Попова Ирина Николаевна,NaN,NaN,NaN,NaN,Троицко-Печорский район,13,Республика Коми,"Муниципальный, без структуры",Республика Коми,12. Попова Ирина Николаевна


In [68]:
# выгрузим в эксель и просмотрим, что это за чиновники
# df2020[df2020['size'].duplicated(keep=False)].to_excel('dubli3.xlsx')

In [42]:
# вносим индексы в список
indexes_excl2020_4 = df2020[df2020[['size', 'region']].duplicated()].index.to_list()
len(indexes_excl2020_4)

413

In [43]:
# убираем дубли из итогового датафрейма
DFcheck.loc[DFcheck.index.isin(indexes_excl2020_4), 'region'] = None

In [44]:
# выборочно посмотрим, что по дублям стоит None в столбце region
DFcheck[DFcheck['size']==1191041.15]

,index,size,comment,section_id,income_year,original_fio,full_name,person_id,position,department,office_name,region_id,region_name,name,region,person_name
130625,130731,1191041.15,NaN,3781817,2020,Попова Ирина Николаевна,NaN,NaN,(29.12.1972 г.р.) директор МБОДО Троицко-Печор...,NaN,Троицко-Печорский район,13,Республика Коми,"Муниципальный, без структуры",Республика Коми,Попова Ирина Николаевна
130649,130755,1191041.15,NaN,3782029,2020,12. Попова Ирина Николаевна,NaN,NaN,NaN,NaN,Троицко-Печорский район,13,Республика Коми,"Муниципальный, без структуры",None,12. Попова Ирина Николаевна


# Final breakdown

In [45]:
# False - нет региона, True - есть регион
# количество увеличилось за счет того, что повторы теперь без региона
pd.notna(DFcheck['region']).value_counts()

True     116922
False     14825
Name: region, dtype: int64

In [46]:
# сгруппируем по региону
reg2020 = DFcheck[pd.notna(DFcheck['region'])].groupby('region').agg(['mean', 'median'])['size'].astype(int)
reg2020

,mean,median
region,,
Алтайский край,1391738,652949
Амурская область,1550634,1051891
Архангельская область,1490656,1086962
Астраханская область,1191773,740773
Белгородская область,1312072,750051
...,...,...
Чеченская республика,5165171,1013285
Чувашская республика - Чувашия,1907672,906109
Чукотский автономный округ,3232289,2164334


In [47]:
# рассчитаем средний доход и медиану помесячно и в тысячах рублей (изначально - суммы за год)
reg2020['mean_mnth_thnds'] = (reg2020['mean']/12/1000).astype(int)
reg2020['median_mnth_thnds'] = (reg2020['median']/12/1000).astype(int)
reg2020

,mean,median,mean_mnth_thnds,median_mnth_thnds
region,,,,
Алтайский край,1391738,652949,115,54
Амурская область,1550634,1051891,129,87
Архангельская область,1490656,1086962,124,90
Астраханская область,1191773,740773,99,61
Белгородская область,1312072,750051,109,62
...,...,...,...,...
Чеченская республика,5165171,1013285,430,84
Чувашская республика - Чувашия,1907672,906109,158,75
Чукотский автономный округ,3232289,2164334,269,180


In [48]:
# Выгружаем данные по регионам в эксель для последующего анализа.
reg2020.to_excel('reg2020.xlsx')

In [60]:
# Расчет для текстовой версии (для размещения на сайт).
# Медианнный доход чиновника по России в целом, тыс.руб.
round(DFcheck[pd.notna(DFcheck['region'])]['size'].median()/12/1000)

82

In [82]:
# Расчет для текстовой версии (для размещения на сайт).
# Доля антикоррупционных деклараций, вошедших в итоговый расчет,
# т.к. не все сведения о доходах удалось отнести к конкретному региону.
100 * (pd.notna(DFcheck['region']).value_counts()[1] / pd.notna(DFcheck['region']).count())

88.74737185666467

Сопоставление медианных доходов чиновников и жителей по регионам сделано в эксель (данных немного, удобнее). Визуализация - в сервисе Datawrapper (https://www.datawrapper.de/).